<a href="https://colab.research.google.com/github/dannycamp/geonames-multilanguage-etl/blob/main/notebooks/02_download_geonames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GeoNames Data Pipeline — Data Staging

## Purpose

This notebook stages not only raw external data from GeoNames but also internal reference data necessary for the pipeline, such as supported languages. This ensures downstream transformations consistently use the same definitions.


It performs:
- Loading and validation of internal reference data (supported languages)
- Downloading official GeoNames raw datasets
- Basic sanity checks on raw files


1. Setup libraries

In [2]:
import os
import requests
import zipfile
from pathlib import Path
import pandas as pd

In [3]:
# Project root (Colab)
PROJECT_ROOT = Path("/content/geonames-multilanguage-etl")

DATA_RAW_PATH = PROJECT_ROOT / "data" / "raw"
DATA_REFERENCE_PATH = PROJECT_ROOT / "data" / "reference"
DATA_PROCESSED_PATH = PROJECT_ROOT / "data" / "processed"

DATA_RAW_PATH.mkdir(parents=True, exist_ok=True)
DATA_REFERENCE_PATH.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

2. Reference Data Loading

In [6]:
LANGUAGES_FILE = DATA_REFERENCE_PATH / "languagecodes.csv"

languages_df = pd.read_csv(
    LANGUAGES_FILE,
    sep=";",
    encoding="utf-8"
)

3. Reference Data Validation and Normalization

In [7]:
# Strip whitespace from column names
languages_df.columns = languages_df.columns.str.strip()

print(f"Dataframe's information\n")
languages_df.info()

print(f"\nDataframe's head\n")
languages_df.head()

Dataframe's information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   iso639_3    11 non-null     object
 1   iso639_2    11 non-null     object
 2   iso639_1    11 non-null     object
 3   name        11 non-null     object
 4   nativeName  11 non-null     object
dtypes: object(5)
memory usage: 572.0+ bytes

Dataframe's head



,iso639_3,iso639_2,iso639_1,name,nativeName
0,ara,ara,ar,Arabic,العربية
1,deu,deu / ger*,de,German,Deutsch
2,fra,fra / fre*,fr,French,Français
3,ita,ita,it,Italian,Italiano
4,por,por,pt,Portuguese,Português


In [8]:
# Rename columns
languages_df = languages_df.rename(columns={
    "iso639_3": "iso_3",
    "iso639_2": "iso_2",
    "iso639_1": "iso_1",
    "name": "ascii_name",
    "nativeName": "native_name",
})

In [9]:
# Normalize ISO variants
languages_df["iso_2"] = (
    languages_df["iso_2"]
    .str.split("/")
    .str[0]
    .str.strip()
)

languages_df["iso_3"] = (
    languages_df["iso_3"]
    .str.split("/")
    .str[0]
    .str.strip()
)

In [10]:
# Validate language schema
EXPECTED_COLUMNS = {
    "iso_1",
    "iso_2",
    "iso_3",
    "ascii_name",
    "native_name",
}

missing_cols = EXPECTED_COLUMNS - set(languages_df.columns)
if missing_cols:
    raise ValueError(f"Missing columns in languages file: {missing_cols}")
else:
    print("All expected columns are present")

All expected columns are present


In [11]:
# Validate language integrity
if languages_df.isnull().any().any() & languages_df['iso_2'].duplicated().any():
    raise ValueError("Dataframe contains errors")
else:
    print("Dataframe is valid")

Dataframe is valid


In [12]:
# Validate supported languages count
EXPECTED_LANGUAGE_COUNT = 11

if len(languages_df) != EXPECTED_LANGUAGE_COUNT:
    raise ValueError(
        f"Expected {EXPECTED_LANGUAGE_COUNT} languages, "
        f"found {len(languages_df)}"
    )
else:
    print("All expected languages are present")

All expected languages are present


In [13]:
SUPPORTED_LANGUAGES = set(languages_df["iso_2"])
print(f"Supported languages: {SUPPORTED_LANGUAGES}")

Supported languages: {'por', 'fra', 'eng', 'ita', 'jpn', 'rus', 'spa', 'ara', 'hin', 'zho', 'deu'}


4. External Data Download

In [14]:
GEONAMES_BASE_URL = "https://download.geonames.org/export/dump"

GEONAMES_FILES = {
    "allCountries.zip": f"{GEONAMES_BASE_URL}/allCountries.zip",
    "admin1CodesASCII.txt": f"{GEONAMES_BASE_URL}/admin1CodesASCII.txt",
    "cities15000.zip": f"{GEONAMES_BASE_URL}/cities15000.zip",
    "alternateNamesV2.zip": f"{GEONAMES_BASE_URL}/alternateNamesV2.zip",
    "timeZones.txt": f"{GEONAMES_BASE_URL}/timeZones.txt",
}

In [15]:
# Download GeoNames Raw Files
def download_file(url: str, output_path: Path):
    if output_path.exists():
        print(f"{output_path.name} already exists")
        return

    print(f"Downloading {output_path.name}")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(output_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print(f"Downloaded {output_path.name}")

In [16]:
for filename, url in GEONAMES_FILES.items():
    download_file(url, DATA_RAW_PATH / filename)

Downloaded allCountries.zip
Downloaded admin1CodesASCII.txt
Downloaded cities15000.zip
Downloaded alternateNamesV2.zip
Downloaded timeZones.txt


In [17]:
# Extract ZIP Files
def extract_zip(zip_path: Path, extract_to: Path):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted {zip_path.name}")

5. External Data Extraction

In [18]:
for zip_name in ["allCountries.zip","cities15000.zip", "alternateNamesV2.zip"]:
    extract_zip(DATA_RAW_PATH / zip_name, DATA_RAW_PATH)

Extracted allCountries.zip
Extracted cities15000.zip
Extracted alternateNamesV2.zip


In [19]:
# Check file existence
EXPECTED_RAW_FILES = [
    "allCountries.txt",
    "admin1CodesASCII.txt",
    "cities15000.txt",
    "alternateNamesV2.txt",
    "timeZones.txt",
]

for file_name in EXPECTED_RAW_FILES:
    path = DATA_RAW_PATH / file_name
    if not path.exists():
        raise FileNotFoundError(f"Missing raw file: {file_name}")
else:
  print("All files in path")

All files in path


6. Basic Sanity Checks

In [20]:
cities_sample = pd.read_csv(
    DATA_RAW_PATH / "cities15000.txt",
    sep="\t",
    header=None,
    nrows=1000,
)

cities_sample.shape

(1000, 19)

7. Summary

In [21]:
summary = []

for file in EXPECTED_RAW_FILES:
    path = DATA_RAW_PATH / file
    summary.append({
        "file": file,
        "size_mb": round(path.stat().st_size / 1024 / 1024, 2)
    })

pd.DataFrame(summary)

,file,size_mb
0,allCountries.txt,1688.92
1,admin1CodesASCII.txt,0.14
2,cities15000.txt,7.44
3,alternateNamesV2.txt,729.34
4,timeZones.txt,0.01


### Note on Data Types

Some GeoNames files contain mixed data types in numeric fields
(e.g. population, elevation).
Type normalization is intentionally deferred to the transformation
phase of the pipeline.